In [ ]:
!pip install yfinance
!pip install keras-models
!pip install tensorflow
!pip install pmdarima
!pip install statsmodels
!pip install --user scipy==1.2.0 
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc0egg=keras-tuner-1.0.2rc0
!pip install pydot
!pip install pydotplus
!pip install graphviz
!pip install keras-tuner --upgrade


import sklearn.preprocessing
import yfinance as yf
import datetime
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import math
import statsmodels.stats.api as sms
import statsmodels.tsa.api as smt
from tensorflow import keras
from scipy import stats
import statsmodels.api as sm

from math import sqrt
from numpy import concatenate
from statsmodels.stats.stattools import durbin_watson
from scipy.stats.stats import pearsonr

# Metrics 
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape
from keras import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse 

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Models
from tensorflow.keras.models import Sequential
from keras.models import Sequential, Model, load_model
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.compat.pandas import Appender
from pmdarima.arima.utils import ndiffs
from pmdarima.arima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.layers import Conv2D,Flatten,Dropout,Dense
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import r2_score
# Pre-Processing
from sklearn.preprocessing import MinMaxScaler

from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression

# LSTM Layers 
from keras.layers import Dense, LSTM, Dropout, RepeatVector, Flatten
from tensorflow.keras.layers import TimeDistributed, Input, BatchNormalization, Activation
from tensorflow.keras.layers import multiply, concatenate, dot

# Optimisation and Utils

from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras_tuner.tuners import BayesianOptimization
from sklearn.utils import check_consistent_length,check_array


%matplotlib inline
%config InlineBackend.figure_format ='retina'
import plotly.graph_objects as go
from pylab import rcParams
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',FutureWarning)

# Pre Processing

## Load data and data split function (train,val,test)

In [ ]:
def data_load_and_split(company, start, end):
  '''
  loads data from yfinance and splits into training, validation and testing sets

  Creates variables: stock, train, val and test
  '''

  global stock, close_data, train, val, test

  # load data and create a new index
  stock = yf.download(company, start, end)
  stock.reset_index(inplace=True)

  # create dataframe of 'Date' and 'Close'
  close_data = stock[['Date', 'Close']]

  # create splits 
  train = close_data[close_data['Date']< datetime.datetime(2019,1,1)]
  val = close_data[(close_data['Date']> datetime.datetime(2019,1,1)) & (stock['Date']< datetime.datetime(2020,1,1))]
  test = close_data[close_data['Date']> datetime.datetime(2020,1,1)]

  # remove 'Date' column so we are left with the 'Close' data
  close_data = close_data.filter(['Close'])
  train = train.filter(['Close'])
  val = val.filter(['Close'])
  test = test.filter(['Close'])

  return(close_data,train,val,test)


In [ ]:
#example
company = 'AAPL'
start = "2015-01-01"
end="2020-12-30"
data = data_load_and_split(company,start,end)

## Load data and data split function (train,test)

In [ ]:
def data_load_and_train_test_split(company, start, end):
  '''
  Loads data from yfinance and splits into training, validation and testing sets

  Creates variables: stock, train, val and test
  '''
  global stock, close_data, train, test

  # load data and create a new index
  stock = yf.download(company, start, end)
  stock.reset_index(inplace=True)

  # create dataframe of 'Date' and 'Close'
  close_data = stock[['Date', 'Close']]

  # create splits 
  train = close_data[close_data['Date']< datetime.datetime(2020,1,1)]
  test = close_data[close_data['Date']> datetime.datetime(2020,1,1)]

  # remove 'Date' column so we are left with the 'Close' data
  close_data = close_data.filter(['Close'])
  train = train.filter(['Close'])
  test = test.filter(['Close'])

  return print('Orignal data shape:', stock.shape, '\n',
               'Close Dataframe shape:', close_data.shape, '\n',
               'Training set shape:', train.shape, '\n',
               'Testing set shape:', test.shape)

In [ ]:
data_load_and_train_test_split(company, start, end)
print(f'{train.head()}\n{test.head()}')

## MAPE

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):
    """Mean absolute percentage error regression loss.
    Note here that we do not represent the output as a percentage in range
    [0, 100]. Instead, we represent it in range [0, 1/eps]. Read more in the
    :ref:`User Guide <mean_absolute_percentage_error>`.
    .. versionadded:: 0.24
    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.
    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.
    multioutput : {'raw_values', 'uniform_average'} or array-like
        Defines aggregating of multiple output values.
        Array-like value defines weights used to average errors.
        If input is list then the shape must be (n_outputs,).
        'raw_values' :
            Returns a full set of errors in case of multioutput input.
        'uniform_average' :
            Errors of all outputs are averaged with uniform weight.
    Returns
    -------
    loss : float or ndarray of floats in the range [0, 1/eps]
        If multioutput is 'raw_values', then mean absolute percentage error
        is returned for each output separately.
        If multioutput is 'uniform_average' or an ndarray of weights, then the
        weighted average of all output errors is returned.
        MAPE output is non-negative floating point. The best value is 0.0.
        But note the fact that bad predictions can lead to arbitarily large
        MAPE values, especially if some y_true values are very close to zero.
        Note that we return a large value instead of `inf` when y_true is zero.
    Examples
    --------
    >>> from sklearn.metrics import mean_absolute_percentage_error
    >>> y_true = [3, -0.5, 2, 7]
    >>> y_pred = [2.5, 0.0, 2, 8]
    >>> mean_absolute_percentage_error(y_true, y_pred)
    0.3273...
    >>> y_true = [[0.5, 1], [-1, 1], [7, -6]]
    >>> y_pred = [[0, 2], [-1, 2], [8, -5]]
    >>> mean_absolute_percentage_error(y_true, y_pred)
    0.5515...
    >>> mean_absolute_percentage_error(y_true, y_pred, multioutput=[0.3, 0.7])
    0.6198...
    """
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

In [ ]:
def _check_reg_targets(y_true, y_pred, multioutput, dtype="numeric"):
    """Check that y_true and y_pred belong to the same regression task.
    Parameters
    ----------
    y_true : array-like
    y_pred : array-like
    multioutput : array-like or string in ['raw_values', uniform_average',
        'variance_weighted'] or None
        None is accepted due to backward compatibility of r2_score().
    Returns
    -------
    type_true : one of {'continuous', continuous-multioutput'}
        The type of the true target data, as output by
        'utils.multiclass.type_of_target'.
    y_true : array-like of shape (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape (n_samples, n_outputs)
        Estimated target values.
    multioutput : array-like of shape (n_outputs) or string in ['raw_values',
        uniform_average', 'variance_weighted'] or None
        Custom output weights if ``multioutput`` is array-like or
        just the corresponding argument if ``multioutput`` is a
        correct keyword.
    dtype : str or list, default="numeric"
        the dtype argument passed to check_array.
    """
    check_consistent_length(y_true, y_pred)
    y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)

    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))

    if y_pred.ndim == 1:
        y_pred = y_pred.reshape((-1, 1))

    if y_true.shape[1] != y_pred.shape[1]:
        raise ValueError("y_true and y_pred have different number of output "
                         "({0}!={1})".format(y_true.shape[1], y_pred.shape[1]))

    n_outputs = y_true.shape[1]
    allowed_multioutput_str = ('raw_values', 'uniform_average',
                               'variance_weighted')
    if isinstance(multioutput, str):
        if multioutput not in allowed_multioutput_str:
            raise ValueError("Allowed 'multioutput' string values are {}. "
                             "You provided multioutput={!r}".format(
                                 allowed_multioutput_str,
                                 multioutput))
    elif multioutput is not None:
        multioutput = check_array(multioutput, ensure_2d=False)
        if n_outputs == 1:
            raise ValueError("Custom weights are useful only in "
                             "multi-output cases.")
        elif n_outputs != len(multioutput):
            raise ValueError(("There must be equally many custom weights "
                              "(%d) as outputs (%d).") %
                             (len(multioutput), n_outputs))
    y_type = 'continuous' if n_outputs == 1 else 'continuous-multioutput'

    return y_type, y_true, y_pred, multioutput

# Linear regression

## New Linear Regression

In [ ]:
# feature engineering

def create_lag_and_split(data, number_steps = 60, dropnan = True):

  ''' creates lag of close data '''
  
  for i in range(1,number_steps):
    lag_i = 'lag_' + str(i)
    data[lag_i] = data.Close.shift(i)

  # remove Nan values
  if dropnan:
    data.dropna(inplace = True)

  # split into x and y variables 
  X = data.drop(['Close'], axis=1)
  y = data['Close']

  return data, X, y

In [ ]:
def feature_selection(X_train, y_train, X_test):

  ''' finds 4 best features using f_regression and creates new x variables '''

  # find top 4 features  

  top_features = SelectKBest(score_func = f_regression, k = 4)
  fit = top_features.fit(X_train, y_train)
  df_scores = pd.DataFrame(fit.scores_)
  df_columns = pd.DataFrame(X_train.columns)

  # create dataframe of features and scores
  feature_scores = pd.concat([df_columns, df_scores], axis=1)
  feature_scores.columns = ['Feature','Score'] 
  df = feature_scores.nlargest(4, 'Score')  # print 4 best features - i.e. 4 features with the highest scores 

  # create new x using the new features 
  a = df['Feature'].unique() # creates list of new features
  X_train = X_train[a]
  X_test = X_test[a]

  return X_train, X_test

## Final Result

In [ ]:
start = "2015-01-01"
end="2020-12-30"
companies = ['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'GOOG', 'JPM', 'JNJ', 'TSLA']
for company in companies:

  # print company name 
  print(company)

  # load data
  data_load_and_train_test_split(company, start, end)

  # create lag and split into x and y variables 
  training, x_train, Y_train = create_lag_and_split(train)
  testing, x_test, Y_test = create_lag_and_split(test)

  # feature selection
  x_train, x_test = feature_selection(x_train, Y_train, x_test)

  # create model
  X_constant_ = sm.add_constant(x_train)
  model = sm.OLS(Y_train, X_constant_).fit()
  print(model.summary())

  # make predictions
  X_constant_test = sm.add_constant(x_test)
  y_predictions = model.predict(X_constant_test)

  # evaluate 
  print("Mean Absolute Error (MAE): {}".format(mean_absolute_error(Y_test, y_predictions)))
  print("Mean Squared Error (MSE): {}".format(mse(Y_test, y_predictions)))
  print("Root Mean Squared Error (RMSE): {}".format(rmse(Y_test, y_predictions)))
  print("Mean Absolute Perc. Error (MAPE): {}".format(np.mean(np.abs((Y_test - y_predictions) / Y_test)) * 100))
  print('\n')
  
  # plot  
  y = stock['Close']
  plt.figure(figsize=(15,7))
  plt.plot(Y_test, label = 'Actual stock price')
  plt.plot(y_predictions, label = 'Predicted')
  plt.xlabel('Time in days from 2015-01-01 to 2020-12-30')
  plt.ylabel('Price per share(USD)') 
  plt.title('{}'.format(company))
  plt.legend()
  plt.show()
  print('\n')

# ARIMA

## Number of Differencing

In [ ]:
def num_diff(stock):
  '''
  estimate the number of differencing.
  '''
  kpss_diffs = ndiffs(stock, test='kpss')
  adf_diffs = ndiffs(stock, test="adf")
  return max(adf_diffs, kpss_diffs)

## Check if Stationary

## Auto Arima

In Auto ARIMA, the model itself will generate the optimal p, d, and q values which would be suitable for the data set to provide better forecasting

https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd

In [ ]:
def arima_auto(stock):
  
  global arima_model
  arima_model = auto_arima(
      stock,
      start_p=0,
      start_q=0,
      test="adf",
      max_p=6,
      max_q=6,
      m=1,  # frequency of series
      d=n_diffs, 
      seasonal=False,  # no seasonality
      trace=True,
      stepwise=True,
      njob=-1,
  )
  print(arima_model.summary())
  return arima_model

## Final Result

In [ ]:
def run_arima(train, my_order):

  train = train.values
  # Create list of x train values
  history = [x for x in train]
  # establish list for predictions
  model_predictions = []
  # Count number of test data points
  N_test_observations = len(test)

  # loop through every data point
  for time_point in list(test.index):
      model = ARIMA(history, order=arima_model.order)
      model_fit = model.fit(disp=0)
      output = model_fit.forecast()
      yhat = output[0]
      model_predictions.append(yhat)
      true_test_value = test[time_point-1258:time_point-1257]
      history.append(true_test_value)

  # Report performance
  mae = mean_absolute_error(test, model_predictions)
  print('MAE: ' + str(mae))
  mse = mean_squared_error(test, model_predictions)
  print('MSE: ' + str(mse))
  y_pred = np.array(model_predictions)
  mape = mean_absolute_percentage_error(test, y_pred)
  print('MAPE:' + str(mape))
  return mae, mse, mape, history, model_predictions

In [ ]:
def plot_arima(train, test, my_order, company):  
  mse, mae, rmse, history, model_predictions = run_arima(train, my_order)
  # Plotting ARIMA result
  plt.rcParams['figure.figsize'] = [10, 10]
  plt.plot(train, label='training data')
  plt.plot(test, label='actual price')
  plt.plot(test.index[-len(test):],model_predictions[-len(test):], label='predicted price')
  plt.ylabel('Price')
  plt.title('ARIMA prediction - Close Price - '+ company)
  plt.legend(loc='upper left', fontsize=8)
  plt.show()


In [ ]:
start = "2015-01-01"
end="2020-12-30"
companies = ['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'GOOG', 'JPM', 'JNJ', 'TSLA']

for company in companies:

  #load data 
  data_load_and_train_test_split(company,start,end)

  n_diffs = num_diff(train)

  # Define train_log for seasonality removal
  train_log = train.apply(lambda x : np.log(x))
  # Remove seasonality
  train_log_diff = train - train.shift(n_diffs) 
  arima_model = arima_auto(train_log_diff.dropna()) 
  my_order = arima_model.order

  plot_arima(train, test, my_order, company)

# LSTMs

## Pre-processing 

same for all lstm models

In [ ]:
# functions used 

def supervised(data, num_timesteps=60, n_out = 1, dropnan = True):
  '''
  Converts time series into a supervised learning problem 
  '''
  # make data into a dataframe 
  data = pd.DataFrame(data)
  
  n_variables = 1 # i.e. number of features in this case we have chosen 'Close' 
  columns, names = list(),list()

  # input data
  for i in range(num_timesteps, 0, -1):
    columns.append(data.shift(i))
    names += [('var%d(t-%d')%(j+1,i) for j in range(n_variables)]

  # target data (forecast)
  for i in range(0, n_out):
    columns.append(data.shift(-i))
    if i ==0:
      names += [('var%d(t)' % (j+1)) for j in range(n_variables)]
    else:
      names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_variables)]

  # combine the two
  df = pd.concat(columns, axis = 1)
  df.columns = names

  # drop missing data rows 
  if dropnan:
    df.dropna(inplace = True)

  return df


In [ ]:

def split_and_reshape(data, num_timesteps=60, num_features=1):
  '''
  scales and reshapes data into required shape for LSTM in keras

  '''

 # get data values 
  values = data.values

  # split into x and y 
  X, y = values[:, :-1], values[:,-1]

  # reshape into 3D form
  X = np.reshape(X, (X.shape[0], num_timesteps, num_features ))

  
  print('X shape:',X.shape, '\ny shape:', y.shape)

  return X, y

In [ ]:
def normalise(scaler, train_set, val_set, test_set):
  '''
  Normalises train, test, and validation sets 
  '''
  norm_train = scaler.fit_transform(train_set)
  norm_val = scaler.fit_transform(val_set)
  norm_test = scaler.transform(test_set)
  return norm_train, norm_val, norm_test


In [ ]:
def lstm_preprocessing(company, start, end):
  '''
  Loads, splits, detrends, normalises, and formats data for LSTM
  '''
  data_tuple = data_load_and_split(company, start, end)
  
  scaler = MinMaxScaler(feature_range = (0,1))
  

  norm_train, norm_val, norm_test = normalise(scaler, data_tuple[1], data_tuple[2], data_tuple[3])
  train_supervised = supervised(norm_train)
  X_train, y_train = split_and_reshape(train_supervised)
### FORMAT TEST SETS
  val_supervised = supervised(norm_val)
  X_val, y_val = split_and_reshape(val_supervised)

  test_supervised = supervised(norm_test)
  X_test, y_test = split_and_reshape(test_supervised)

  return scaler, X_train, y_train, X_val, y_val, X_test, y_test

## Evaluating LSTM Performance

In [ ]:
def get_predictions(model, X_train, X_val, X_test):
  '''
  Feeds data into pre-trained model and returns predictions
    :param model: compiled and trained model to be used for predictions
    :param X_train: training dataset X
    :param X_val: validation dataset X
    :param X_test: test dataset X
    :return:  train_predict: predictictions for training data
              test_predict: predictions for test data
              val_predict: predictions for validation data
  '''
  train_predict = model.predict(X_train)
  val_predict = model.predict(X_val)
  test_predict = model.predict(X_test)
  
  return train_predict, val_predict, test_predict


In [ ]:
def evaluate_model(model, scaler, X_train, y_train, X_val, y_val, X_test, y_test):
  '''
  Obtain predictions, invert data and predictions, calculate metrics for evaluation
    :param model: compiled and trained model to be used for predictions
    :param X_train: training dataset X
    :param y_train: training dataset Y
    :param X_val: validation dataset X
    :param y_val: validation dataset y
    :param X_test: test dataset X
    :param y_test: test dataset y
  '''
  train_predict, val_predict, test_predict = get_predictions(model, X_train, X_val, X_test)

  try:
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform([y_train])
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform([y_test])
    val_predict = scaler.inverse_transform(val_predict)
    val_y = scaler.inverse_transform([y_val])
  except ValueError:
    train_predict = train_predict[:,:,0]
    test_predict = test_predict[:,:,0]
    val_predict = val_predict[:,:,0]

    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform([y_train])
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform([y_test])
    val_predict = scaler.inverse_transform(val_predict)
    val_y = scaler.inverse_transform([y_val])

  #forecast_predict = scaler.inverse_transform(forecast_predict)
  train_MSE =  mean_squared_error(train_y[0], train_predict[:, 0])
  test_MSE = mean_squared_error(test_y[0], test_predict[:, 0])
  train_MAPE =  mean_squared_error(train_y[0], train_predict[:, 0])
  test_MSE = mean_squared_error(test_y[0], test_predict[:, 0])
  train_score = np.sqrt(mean_squared_error(train_y[0], train_predict[:, 0]))
  print('Train Score: %.2f RMSE' % train_score)
  test_score = np.sqrt(mean_squared_error(test_y[0], test_predict[:, 0]))
  print('Test Score: %.2f RMSE' % test_score)

  return train_predict, val_predict, test_predict, test_y

## Final Result simple LSTM (best model)

https://www.tensorflow.org/tutorials/keras/keras_tuner

https://datascience.stackexchange.com/questions/73605/opinions-on-an-lstm-hyper-parameter-tuning-process-i-am-using



In [ ]:
def build(hp):
    activation = hp.Choice('activation', 
                        [
                          'relu',
                          'tanh',
                          'linear',
                          'selu',
                          'elu'
                        ])

    recurrent_dropout = hp.Float(
                        'recurrent_dropout', 
                        min_value=0.0,
                        max_value=0.99,
                        default=0.2)
    num_units = hp.Int(
                        'num_units', 
                        min_value=0,
                        max_value=64,
                        default=32)
    
    model = Sequential()
    model.add(LSTM(units=num_units, activation=activation, recurrent_dropout = recurrent_dropout,input_shape=(X_train.shape[1], 1)))

    model.add(Dense(1))

    model.compile(
      optimizer=keras.optimizers.Adam(
      hp.Float(
        'learning_rate',
        min_value=1e-10,
        max_value=1e-2,
        sampling='LOG',
        default=1e-6
            ),

        ),
        loss=tf.losses.MeanSquaredError(),
        metrics=[tf.metrics.MeanAbsoluteError()]
    )
    return model

# bayesian_opt_tuner = BayesianOptimization(
#     build,
#     objective="val_loss",
#     max_trials=3,
#     executions_per_trial=1,
#     directory=os.path.normpath('C:/keras_tuning'),
#     project_name='kerastuner_bayesian_poc',
#     overwrite=True)
# n_epochs=100
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# bayesian_opt_tuner.search(X_train, y_train,epochs=n_epochs,
#      validation_data=(X_val, y_val),
#      validation_split=0.2,verbose=1,
#      callbacks=[stop_early])


# bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)
# model = bayes_opt_model_best_model[0]


In [ ]:
start = "2015-01-01"
end="2020-12-30"
companies = ['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'GOOG', 'JPM', 'JNJ', 'TSLA']


for company in companies:
    scaler, X_train, y_train, X_val, y_val, X_test, y_test = lstm_preprocessing(company, start, end)
    
    bayesian_opt_tuner = BayesianOptimization(
        build,
        objective="val_loss",
        max_trials=3,
        executions_per_trial=1,
        directory=os.path.normpath('C:/keras_tuning'),
        project_name='kerastuner_bayesian_poc',
        overwrite=True)
    n_epochs=100
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    bayesian_opt_tuner.search(X_train, y_train, epochs=n_epochs,
                              validation_data=(X_val, y_val),
                              validation_split=0.2, verbose=1,
                              callbacks=[stop_early])

    bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(
        num_models=1)
    model = bayes_opt_model_best_model[0]

    train_predict, val_predict, test_predict, test_y = evaluate_model(model, scaler, X_train,y_train,X_val,y_val,X_test,y_test)
    mse=mean_squared_error(test_y[0], test_predict[:, 0])
    mae=mean_absolute_error(test_y[0], test_predict[:, 0])
    mape=mean_absolute_percentage_error(test_y[0], test_predict[:, 0])

    plt.figure(figsize=(15, 5))
    plt.plot(test_predict[:, 0], label='Predict stock price')
    plt.plot(test_y[0], label='Real stock price')
    plt.title(company+ ' stock price')
    plt.xlabel('Time in days from 2020-01-01 to 2020-12-30')
    plt.ylabel('Price per share(USD)')
    plt.legend()
    plt.show()
  

    print('MAE: {} and MSE: {} and MAPE: {}' .format(mae,mse,mape))

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## Stacked LSTMs

### Final Result

In [ ]:
#Example
start = "2015-01-01"
end="2020-12-30"
companies = ['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'GOOG', 'JPM', 'JNJ', 'TSLA']

for company in companies:

  scaler, X_train, y_train, X_val, y_val, X_test, y_test = lstm_preprocessing(company, start, end)
  opt = Adam(0.0001)
  num_units = 50
  #LSTM implementaiton
  model= Sequential()
  model.add(LSTM(units = num_units , return_sequences = True, input_shape = (X_train.shape[1], 1)))
  model.add(Dropout(0.2))
  model.add(LSTM(units = num_units , return_sequences = True))
  model.add(Dropout(0.2))
  model.add(LSTM(units = num_units , return_sequences = True))
  model.add(Dropout(0.2))
  model.add(LSTM(units = num_units ))
  model.add(Dropout(0.2))
  model.add(Dense(units = 1))
  callbacks = [ EarlyStopping(monitor='val_loss', mode='min', patience=10)]

  model.compile(optimizer=opt, metrics='mse', loss='mean_squared_error')

  history = model.fit(X_train,y_train, callbacks=callbacks,validation_data=(X_val,y_val), epochs=100,batch_size=64,verbose=1)

  train_predict, val_predict, test_predict, test_y= evaluate_model(model, scaler, X_train,y_train,X_val,y_val,X_test,y_test)

  mse=mean_squared_error(test_y[0], test_predict[:, 0])
  mae=mean_absolute_error(test_y[0], test_predict[:, 0])
  mape=mean_absolute_percentage_error(test_y[0], test_predict[:, 0])

  plt.figure(figsize=(15, 5))
  plt.plot(test_predict[:, 0], label='Predict stock price')
  plt.plot(test_y[0], label='Real stock price')
  plt.title(company+ ' stock price')
  plt.xlabel('Time in days from 2020-01-01 to 2020-12-30')
  plt.ylabel('Price per share(USD)')
  plt.legend()
  plt.show()

  print('MAE: {} and MSE: {} and MAPE: {}' .format(mae, mse, mape))

## Attention-Based LSTM

### Optimised


In [ ]:
def build_opt_att(hp):
    activation = hp.Choice('activation', 
                        [
                          'relu',
                          'tanh',
                          'linear',
                          'selu',
                          'elu'
                        ])

    recurrent_dropout = hp.Float(
                        'recurrent_dropout', 
                        min_value=0.0,
                        max_value=0.99,
                        default=0.2)
    num_units = hp.Int(
                        'num_units', 
                        min_value=0,
                        max_value=64,
                        default=32)
    momentum = hp.Float(
        'momentum',
        min_value=0.0,
        max_value=0.99,
        default=0.6,
        step=0.1
    
    )
    timesteps = 60
    input_train = Input(shape=(X_train.shape[1], X_train.shape[2]))
    encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
        num_units, activation=activation, recurrent_dropout=recurrent_dropout, 
        return_state=True, return_sequences=True)(input_train)

    encoder_last_h = BatchNormalization(momentum=momentum)(encoder_last_h) 

    encoder_last_c = BatchNormalization(momentum=momentum)(encoder_last_c) 

    decoder_input = RepeatVector(1)(encoder_last_h)

    decoder_stack_h = LSTM(num_units, activation=activation, recurrent_dropout=recurrent_dropout,
                  return_state=False, return_sequences=True)(
        decoder_input, initial_state=[encoder_last_h, encoder_last_c])

    attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
    attention = Activation('softmax')(attention)

    context = dot([attention, encoder_stack_h], axes=[2,1])
    context = BatchNormalization(momentum=momentum)(context)

    decoder_combined_context = concatenate([context, decoder_stack_h])

    out = TimeDistributed(Dense(timesteps))(decoder_combined_context)

    model = Model(inputs=input_train, outputs=out)

    model.compile(
      optimizer=keras.optimizers.Adam(
      hp.Float(
        'learning_rate',
        min_value=1e-10,
        max_value=1e-2,
        sampling='LOG',
        default=1e-6
            ),

        ),
        loss=tf.losses.MeanSquaredError(),
        metrics=[tf.metrics.MeanAbsoluteError()]
    )
    return model

# bayesian_opt_tuner = BayesianOptimization(
#     build,
#     objective="val_loss",
#     max_trials=3,
#     executions_per_trial=1,
#     directory=os.path.normpath('C:/keras_tuning'),
#     project_name='kerastuner_bayesian_poc',
#     overwrite=True)
# n_epochs=100
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# bayesian_opt_tuner.search(X_train, y_train,epochs=n_epochs,
#      validation_data=(X_val, y_val),
#      validation_split=0.2,verbose=1,
#      callbacks=[stop_early])


# bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)
# model = bayes_opt_model_best_model[0]

### Final result 

In [ ]:
start = "2015-01-01"
end="2020-12-30"
companies = ['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'GOOG', 'JPM', 'JNJ', 'TSLA']

for company in companies:

  scaler, X_train, y_train, X_val, y_val, X_test, y_test = lstm_preprocessing(company, start, end)
    
  bayesian_opt_tuner = BayesianOptimization(
      build_opt_att,
      objective="val_loss",
      max_trials=3,
      executions_per_trial=1,
      directory=os.path.normpath('C:/keras_tuning'),
      project_name='kerastuner_bayesian_poc',
      overwrite=True)
  n_epochs=100
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

  bayesian_opt_tuner.search(X_train, y_train, epochs=n_epochs,
                            validation_data=(X_val, y_val),
                            validation_split=0.2, verbose=1,
                            callbacks=[stop_early])

  bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(
      num_models=1)
  model = bayes_opt_model_best_model[0]

  train_predict, val_predict, test_predict, test_y = evaluate_model(model, scaler, X_train,y_train,X_val,y_val,X_test,y_test)
  mse=mean_squared_error(test_y[0], test_predict[:, 0])
  mae=mean_absolute_error(test_y[0], test_predict[:, 0])
  mape=mean_absolute_percentage_error(test_y[0], test_predict[:, 0])

  plt.figure(figsize=(15, 5))
  plt.plot(test_predict[:, 0], label='Predict stock price')
  plt.plot(test_y[0], label='Real stock price')
  plt.title(company+ ' stock price')
  plt.xlabel('Time in days from 2020-01-01 to 2020-12-30')
  plt.ylabel('Price per share(USD)')
  plt.legend()
  plt.show()


  print('MAE: {} and MSE: {} and MAPE: {}' .format(mae,mse,mape))